In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import numpy as np

from pathlib import Path
import re

import itertools
sns.set_style('white')
sns.set_context('paper',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

datadir = Path(Path('datadir.txt').read_text())

In [ ]:
if (datadir/'projects'/'tangles'/'simulations'/'fig1_fig2_summary.gzip').exists():
    summary_df = pd.read_parquet(datadir/'projects'/'tangles'/'simulations'/'fig1_fig2_summary.gzip')
    print(f'Loaded {len(summary_df)} total simulation runs from Parquet cached file')
else:
    summary_df = []
    with h5py.File(
        datadir/'projects'/'tangles'/'simulations'/'modeling_fig1_fig2_combined_summaries.h5', 'r') as summaries:
        for group in summaries.keys():
            group_df = []
            for run in summaries[group].values():
                group_df.append(pd.DataFrame(data={
                    'inducible_mRNA': run['final_mRNA'][0,:],
                    'constant_mRNA': run['final_mRNA'][1,:]}))
                group_df[-1]['induction'] = run.attrs['gene.base_rate'][0]
                group_df[-1]['condition'] = run.attrs['comment'].decode('utf-8')
                group_df[-1]['is_plasmid'] = run.attrs['bcs.is_circular']
                for term in [
                    'rates.sc_dependent', 'coeff.sigma_squared',
                    'coeff.mRNA_drag_exponent', 'coeff.mRNA_drag_coeff',
                    'rnap.max_velocity', 'rnap.stall_torque', 'rnap.stall_width']:
                    group_df[-1][term] = run.attrs[term]
            summary_df.append(pd.concat(group_df, ignore_index=True))
            print(f'Loaded {len(summary_df[-1])} simulations...')
    summary_df = pd.concat(summary_df, ignore_index=True)
    print(f'Loaded {len(summary_df)} total simulation runs!')
    summary_df['condition'] = pd.Categorical(summary_df['condition'])
    summary_df.to_parquet(datadir/'projects'/'tangles'/'simulations'/'fig1_fig2_summary.gzip', compression='gzip')

## Figure 1

### Visual description
A 160mm x 180mm image (6.30 x 7.08 in).

- **Panel A**: A cartoon of polymerases coming on and off a piece of DNA, showing the binding energy dE, accumulated supercoiling, and the key variables.
- **Panel B**: A diagram of the key tested variables in Figure 1, including circular/linear, different orientations, and delta spacing.
- **Panel C**: Three diagrams that show (normalized) reporter output (y-axis) vs induction (x-axis), with different plots for the different supercoiling initiation values and different curves for each orientation (constant spacing). These plots are all linear BCs.
- **Panel D**: Two subplots which compare linear vs plasmid BCs vs different induction conditions, using one of the sc initiation panels.
- **Panel E**: Three subplots. These are scatterplots (density plots?) of the three conditions (at full induction) that show the noise behavior of the system. Added on is a schematic reminding of the intrinsic/extrinsic noise.
- **Panel F**: Three subplots, showing reporter output as a function of induction for three different spacing levels.

### Caption

## Supplemental Figure 1
### Visual description
???

In [ ]:
df_1c = summary_df[
    (
        summary_df['is_plasmid'] == 0
    ) & (
        #(summary_df['condition'] == 'fig1.tandem_downstream') |
        (summary_df['condition'] == 'fig1.tandem_upstream') |
        (summary_df['condition'] == 'fig1.convergent') |
        (summary_df['condition'] == 'fig1.divergent')
    )
].copy()
df_1c['condition'] = df_1c['condition'].cat.remove_unused_categories()
sns.lineplot(data=df_1c[df_1c['rates.sc_dependent'] == 0.0], x='induction', y='constant_mRNA', hue='condition', alpha=0.5)
plt.show()
print(df_1c[df_1c['rates.sc_dependent']==1.0]['coeff.sigma_squared'].unique())
sns.lineplot(data=df_1c[(df_1c['rates.sc_dependent']==1.0) & (df_1c['coeff.sigma_squared']==0.001)], x='induction', y='constant_mRNA', hue='condition', alpha=0.5)
plt.show()
sns.lineplot(data=df_1c[(df_1c['rates.sc_dependent']==1.0) & (np.abs(df_1c['coeff.sigma_squared']-0.1) < 0.001)], x='induction', y='constant_mRNA', hue='condition', alpha=0.5)
plt.show()

In [ ]:
fig1 = plt.figure(figsize=(6.30,7.08))
fig1_subplots = fig1.subplot_mosaic(
    '''
    AB
    CD
    EF
    '''
)

plt.show(fig1)

## Figure 2

### Visual description
A 160mm x 120mm image (6.30 x 4.72 in).

- **Panel A**: A cartoon of polymerases coming on and off a piece of DNA, showing the binding energy dE, accumulated supercoiling, and the key variables.
- **Panel B**: A diagram of the key tested variables in Figure 1, including circular/linear, different orientations, and delta spacing.
- **Panel C**: Three diagrams that show (normalized) reporter output (y-axis) vs induction (x-axis), with different plots for the different supercoiling initiation values and different curves for each orientation (constant spacing). These plots are all linear BCs.
- **Panel D**: Two subplots which compare linear vs plasmid BCs vs different induction conditions, using one of the sc initiation panels.
- **Panel E**: Three subplots. These are scatterplots (density plots?) of the three conditions (at full induction) that show the noise behavior of the system. Added on is a schematic reminding 

### Caption

In [ ]:
fig2 = plt.figure(figsize=)